# LAB: Árboles de decisión y Bagging


## Introducción

En este lab compararemos el rendimiento de un clasificador de árbol de decisión simple con un clasificador de Bagging. Lo haremos en pocos datasets, empezando por los ofrecidos por Scikit Learn.

Como ya hemos aprendido, el data science es un proceso iterativo. A menudo comenzamos con un modelo muy simple y luego tratamos de mejorar su rendimiento o encontrar mejores modelos para comparar con el inicial.

Este es exactamente el proceso que seguiremos en este laboratorio. Comenzaremos con un modelo simple (Árbol de decisión) y luego compararemos su rendimiento con un modelo Ensamble más complejo.

Usaremos dos datasets:

- para clasificación utilizaremos el [dataset de Cáncer de mama](http://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+(Diagnostic). Trataremos de diagnosticar un cáncer de mama a partir de pocas características del núcleo celular

- para regressión utilizaremos el [dataset de Diabetes](http://web.stanford.edu/~hastie/Papers/LARS/LeastAngle_2002.pdf), donde trataremos de obtener una medida cuantitativa de la progresión de la enfermedad un año después de la línea de base de 10 variables basales.

## Ejercicio

### Requisitos

1. Usando el dataset de Cáncer de mama, inicializar un clasificador de árbol de decisión y utilizar cross_val_score para evaluar su rendimiento
    - Compare el rendimiento con el clasificador de Bagging
    - Usar pipelines y escalado para ver si el rendimiento mejora
    - Explorar el espacio de parámetros con Grid Search
- Usando el dataset de Diabetes, inicializar un Árbol de Regresión y compararlo con un Regresor de Bagging
    - Comparación simple usando cross_val_score
    - Explorar el espacio de parámetros con Grid Search

**Extra:**

- Repetir el análisis anterior para el dataset de IMDB

## Recursos Adicionales

- [Validación cruzada en el ejemplo de Diabetes](http://scikit-learn.org/stable/auto_examples/exercises/plot_cv_diabetes.html)
- [paper del dataset de Diabetes](http://web.stanford.edu/~hastie/Papers/LARS/LeastAngle_2002.pdf)
- [Clasificador Bagging](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.BaggingClassifier.html)
- [Grid Search](http://scikit-learn.org/stable/modules/grid_search.html)

## 1. Dataset de Cáncer de mama
Comenzaremos la comparación con el dataset de cáncer de mama.
Pueden leerlo directamente desde scikit-learn usando la función `load_breast_cancer`.

### 1.a Comparación simple
1. Leer los datos y crear X e y
- Inicializar un árbol de clasificación y usar use cross_val_score para evaluar su rendimiento. Setear la validacion cruzada a 5-folds
- Construir un clasificador de Bagging sobre un clasificador de árbol y usar cross_val_score para evaluar su rendimiento. Setear la validacion cruzada a 5-folds
- ¿Qué resultado es mejor? ¿El resultado es significativamente diferente? ¿Cómo se puede interpretar eso?
> Respuesta: usar el desvío estándar de los resultados de salida de cross_val_score

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from sklearn.datasets import load_breast_cancer

In [3]:
data = load_breast_cancer()

In [4]:
data.keys()

dict_keys(['data', 'target', 'target_names', 'DESCR', 'feature_names', 'filename'])

In [5]:
X = pd.DataFrame(data['data'], columns=data['feature_names'])
y = pd.Series(data['target'])

In [6]:
X.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [7]:
y.value_counts()

1    357
0    212
dtype: int64

In [8]:
y.value_counts()/y.count()

1    0.627417
0    0.372583
dtype: float64

In [9]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split


In [10]:
# partimos en entrenamiento-prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=41)

In [11]:
dt = DecisionTreeClassifier()

In [12]:
def do_cross_val(model):
    scores = cross_val_score(model, X_train, y_train, cv=5, n_jobs=-1)
    return scores.mean(), scores.std()

do_cross_val(dt)

(0.9069882012814501, 0.01738583150896301)

In [13]:
bdt = BaggingClassifier(DecisionTreeClassifier())

In [14]:
do_cross_val(bdt)

(0.9194889826535396, 0.03280172275490484)

### 1.b Grid Search

El Grid search es una excelente manera de mejorar el rendimiento de un clasificador. Vamos a explorar el espacio de parámetros de ambos modelos y ver si podemos mejorar su rendimiento.

1. Inicializar un GridSearchCV con una validación cruzada de 5-fold para el árbol de decisión de clasificación
- Buscar algunos valores de los parámetros para mejorar el resultado del clasificador
- Utilizar todo el conjunto de datos X e y para la prueba
- Comparar el best\_score\_ después de entrenarlo. ¿Es mejor que antes?
> Sí
- ¿Cómo se compara el resultado de la Grid Search del árbol de decisión con la del árbol de Bagging?
> Respuesta: son los mismos (dentro del error), la Grid Search mejoró el resultado del árbol simple
- Inicializar un GridSearchCV con validación cruzada de 5-fold para el clasificador de árbol de Bagging
- Repetir la búsqueda
    - Tener en cuenta que hay que cambiar los nombres de los parámetros para el base_estimator
    - Tener en cuenta que también hay parámetros adicionales para cambiar
    - Tener en cuenta que puede terminar con un espacio de búsqueda muy grande y demandará mucho tiempo
    - Utilizar el parámetro n_jobs para acelerar la búsqueda

- ¿Mejora el resultado para el clasificador de Bagging?
> Sí
- ¿Qué resultado es mejor? ¿El resultado es significativamente diferente? ¿Cómo se puede interpretar esto?
> El clasificador de Bagging resultante del GridSearch es el mejor

In [15]:
from sklearn.model_selection import GridSearchCV

In [16]:
params = {"max_depth": [3,10,None],
          "max_features": [None, "auto"],
          "min_samples_leaf": [1, 5, 10],
          "min_samples_split": [2, 10]
         }
    

gsdt = GridSearchCV(dt, params, n_jobs=-1, cv=5)

In [17]:
gsdt.fit(X_train, y_train)

C:\Users\mbeati\AppData\Local\Continuum\anaconda3\envs\dsdh2\lib\site-packages\sklearn\model_selection\_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=DecisionTreeClassifier(class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort=False, random_state=None,
                                              splitter='best'),
             iid='warn', n_jobs=-1,
             param_grid={'max_depth': [3, 10, None],
                         'max_features': [None, 'auto'],
                         'mi

In [18]:
gsdt.best_params_

{'max_depth': 3,
 'max_features': 'auto',
 'min_samples_leaf': 10,
 'min_samples_split': 10}

In [19]:
gsdt.best_score_

0.9221105527638191

In [20]:
bdt.get_params()

{'base_estimator__class_weight': None,
 'base_estimator__criterion': 'gini',
 'base_estimator__max_depth': None,
 'base_estimator__max_features': None,
 'base_estimator__max_leaf_nodes': None,
 'base_estimator__min_impurity_decrease': 0.0,
 'base_estimator__min_impurity_split': None,
 'base_estimator__min_samples_leaf': 1,
 'base_estimator__min_samples_split': 2,
 'base_estimator__min_weight_fraction_leaf': 0.0,
 'base_estimator__presort': False,
 'base_estimator__random_state': None,
 'base_estimator__splitter': 'best',
 'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                        max_features=None, max_leaf_nodes=None,
                        min_impurity_decrease=0.0, min_impurity_split=None,
                        min_samples_leaf=1, min_samples_split=2,
                        min_weight_fraction_leaf=0.0, presort=False,
                        random_state=None, splitter='best'),
 'bootstrap': True,
 'bootstrap_features': 

In [21]:
params = {"base_estimator__max_depth": [3,10,None],
          "base_estimator__max_features": [None, "auto"],
          "base_estimator__min_samples_leaf": [1, 5, 10],
          "base_estimator__min_samples_split": [2, 10],
          'bootstrap_features': [False, True],
          'max_features': [0.5, 1.0],
          'max_samples': [0.5, 1.0],
          'n_estimators': [5, 15, 40],
         }
    

gsbdt = GridSearchCV(bdt, params, n_jobs=3, cv=5)

In [22]:
gsbdt.fit(X, y)

C:\Users\mbeati\AppData\Local\Continuum\anaconda3\envs\dsdh2\lib\site-packages\sklearn\model_selection\_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=BaggingClassifier(base_estimator=DecisionTreeClassifier(class_weight=None,
                                                                               criterion='gini',
                                                                               max_depth=None,
                                                                               max_features=None,
                                                                               max_leaf_nodes=None,
                                                                               min_impurity_decrease=0.0,
                                                                               min_impurity_split=None,
                                                                               min_samples_leaf=1,
                                                                               min_samples_split=2,
                                                   

In [23]:
gsbdt.best_params_

{'base_estimator__max_depth': 10,
 'base_estimator__max_features': 'auto',
 'base_estimator__min_samples_leaf': 1,
 'base_estimator__min_samples_split': 2,
 'bootstrap_features': True,
 'max_features': 1.0,
 'max_samples': 1.0,
 'n_estimators': 15}

In [24]:
gsbdt.best_score_

0.968365553602812

## 2 Diabetes y regresión

Scikit Learn tiene un dataset de pacientes diabéticos obtenido de este estudio:

http://www4.stat.ncsu.edu/~boos/var.select/diabetes.html
http://web.stanford.edu/~hastie/Papers/LARS/LeastAngle_2002.pdf

Se midieron 442 pacientes diabéticos en 10 variables basales: edad, sexo, índice de masa corporal, presión arterial media y seis mediciones del suero sanguíneo.

El objetivo es una medida cuantitativa de la progresión de la enfermedad un año después de la línea de base.

Repita la comparación anterior entre un DecisionTreeRegressor y una versión de Bagging del mismo.

### 2.a Comparación simple
1. Leer los datos y crear X e y
- Inicializar un Árbol de Regresión y utilizar cross_val_score para evaluar su rendimiento. Establecer la validación cruzada en 5-fold. ¿Qué medida de evaluación utilizará?
> Respuesta: r2
- Construir un Árbol de Regresión con Bagging y usar cross_val_score para evaluar su rendimiento. Establecer la validación cruzada en 5-fold.
- ¿Qué resultado es mejor? ¿El resultado es significativamente diferente? ¿Cómo se puede interpretar esto?
> El Árbol de Regresión con Bagging es mejor   
> Respuesta: usar el desvío estándar de los resultados de la salida cross_val_score

In [25]:
from sklearn.datasets import load_diabetes

data = load_diabetes()
X_full = data['data']
y_full = data['target']

X_train, X_test, y_train, y_test = train_test_split( X_full, y_full, test_size=0.1, random_state=42)


In [26]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor

def do_cross_val(model):
    scores = cross_val_score(model, X_train, y_train, cv=5, n_jobs=-1, scoring='r2')
    return scores.mean(), scores.std()


In [27]:
dtr = DecisionTreeRegressor()
do_cross_val(dtr)

(-0.11253127462120753, 0.14410340747657135)

In [28]:
bdtr = BaggingRegressor(DecisionTreeRegressor())
do_cross_val(bdtr)

(0.3575449737221268, 0.10206075094502898)

### 2.b Grid Search

Repetir el Grid search como arriba:

1. Inicializar un GridSearchCV con 5-fold de validación cruzada para el árbol de Regresión
- Buscar unos pocos valores de los parámetros para mejorar el resultado del regresor
- Utilizar todo el set de datos X e y para la prueba
- Comparar el best\_score\_ después de entrenarlo. ¿Es mejor que antes?
> Sí
- ¿Cómo se compara el resultado del Grid Search del árbol de decisión con la del árbol de Bagging?
> Respuesta: son los mismos (dentro del error), el Grid Search mejoró el resultado del árbol simple
- Inicializar un GridSearchCV con validación cruzada de 5-fold para el árbol de Regresión de Bagging
- Repetir la búsqueda

    - Tener en cuenta que hay que cambiar los nombres de los parámetros para el base_estimator
    - Tener en cuenta que también hay parámetros adicionales para probar
    - Tener en cuenta que puede terminar con un espacio de búsqueda muy grande y demandará mucho tiempo
    - Utilizar el parámetro n_jobs para acelerar la búsqueda

- ¿Mejora el resultado para el Bagging de Regresión?
> Sí
- ¿Qué resultado es mejor? ¿El resultado es significativamente diferente? ¿Cómo se puede interpretar esto?
> El Bagging de Regresión resultante del GridSearch es el mejor

In [29]:
dtr=DecisionTreeRegressor()
dtr.get_params()

{'criterion': 'mse',
 'max_depth': None,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'presort': False,
 'random_state': None,
 'splitter': 'best'}

In [30]:
params = {"splitter": ['best', 'random'],
          "max_depth": [3,5,10,20],
          "max_features": [None, "auto"],
          "min_samples_leaf": [1, 3, 7, 10],
          "min_samples_split": [2, 5, 7]
         }
    

gsdtr = GridSearchCV(dtr, params, n_jobs=-1, cv=5, scoring='r2')

In [31]:
gsdtr.fit(X_train, y_train)

C:\Users\mbeati\AppData\Local\Continuum\anaconda3\envs\dsdh2\lib\site-packages\sklearn\model_selection\_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=DecisionTreeRegressor(criterion='mse', max_depth=None,
                                             max_features=None,
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             presort=False, random_state=None,
                                             splitter='best'),
             iid='warn', n_jobs=-1,
             param_grid={'max_depth': [3, 5, 10, 20],
                         'max_features': [None, 'auto'],
                         'min_samples_leaf': [1, 3, 7, 10],
                         'min_samples_split

In [32]:
gsdtr.best_params_

{'max_depth': 5,
 'max_features': None,
 'min_samples_leaf': 10,
 'min_samples_split': 5,
 'splitter': 'random'}

In [33]:
gsdtr.best_score_

0.34974152589115093

In [34]:
params = {"base_estimator__splitter": ['best', 'random'],
          "base_estimator__max_depth": [3,5,10,20],
          "base_estimator__max_features": [None, "auto"],
          "base_estimator__min_samples_leaf": [1, 3, 7, 10],
          "base_estimator__min_samples_split": [2, 5, 7],
          'bootstrap_features': [False, True],
          'max_features': [0.5, 1.0],
          'max_samples': [0.5, 1.0],
          'n_estimators': [5, 15, 40],
         }
    

gsbdtr = GridSearchCV(bdtr, params, n_jobs=-1, cv=5, scoring='r2')

In [35]:
gsbdtr.fit(X, y)

C:\Users\mbeati\AppData\Local\Continuum\anaconda3\envs\dsdh2\lib\site-packages\sklearn\model_selection\_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=BaggingRegressor(base_estimator=DecisionTreeRegressor(criterion='mse',
                                                                             max_depth=None,
                                                                             max_features=None,
                                                                             max_leaf_nodes=None,
                                                                             min_impurity_decrease=0.0,
                                                                             min_impurity_split=None,
                                                                             min_samples_leaf=1,
                                                                             min_samples_split=2,
                                                                             min_weight_fraction_leaf=0.0,
                                                           

In [36]:
gsbdtr.best_params_

{'base_estimator__max_depth': 20,
 'base_estimator__max_features': None,
 'base_estimator__min_samples_leaf': 1,
 'base_estimator__min_samples_split': 5,
 'base_estimator__splitter': 'random',
 'bootstrap_features': True,
 'max_features': 1.0,
 'max_samples': 1.0,
 'n_estimators': 40}

In [37]:
gsbdtr.best_score_

0.8649924366819647

Comparemos como performan los siguientes modelos:
* El mejor Árbol de regresión obtenido con GridSearch
* El mejor Bagging de árboles de regresión obtenido con GridSearch
* Alguno de los Árboles de regresión base del Bagging de árboles


In [38]:
from sklearn.metrics import r2_score

def eval_score(legend,model):
    model_fit = model.fit(X_train, y_train)
    r2=r2_score(y_test,model_fit.predict(X_test))
    print(legend,r2)

eval_score("Mejor árbol de regresión:",gsdtr.best_estimator_)
eval_score("Mejor Bagging de árboles de regresión:",gsbdtr.best_estimator_)
eval_score("Un árbol base del Bagging:",gsbdtr.best_estimator_.estimators_[0])

Mejor árbol de regresión: 0.44077821994049293
Mejor Bagging de árboles de regresión: 0.5071448242614462
Un árbol base del Bagging: 0.06427183634207367
